# Tasty Bytes : Customer Review Analytics

Tasty Bytes is a global food truck network operating in 15 countries with fleet of 450 trucks. They collect customer reviews to get customer feedback on their food-trucks which come in from multiple sources and span multiple languages. This enables them to better understand the areas which require improvement and drive up customer loyalty along with satisfaction. 

In this notebook, we will look at how we analyze these collated customer reviews using Snowflake Cortex to understand :
  * How likely their customers are to recommend Tasty Bytes food trucks to someone they know 
  * How good their overall experience was
  * How is the customer sentiment across customer base
  * What customers are feel saying about different aspects like food, price etc
  * What are the main issues and how to remedy them

We can write SQL or Python code within Snowflake notebook and quickly compare results with cell-by-cell development and execution. Below, we write a quick SQL statement to view how many reviews we have.

In [ ]:
SELECT COUNT(*) FROM TRUCK_REVIEWS_V;

**Import python packages**

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel. Installing a package is made easy by enabling user to select required pacakges from a list of available pacakges under Packages on the top right corner. Once installed, we can import installed packages as we would in any other notebook.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()


We can preview the reviews we have in the analytics view using Snowpark Dataframe.

In [ ]:
reviews_df = session.table('TRUCK_REVIEWS_V') \
             .filter(F.date_part("year", F.col('DATE')) == 2024)
reviews_df.show()

In [ ]:
create or replace table reviews_2024 as 
select *
from 
    truck_reviews_v
where 
    year(date) = 2024

Below, we look at the distinct languages available within our reviews.

In [ ]:
select distinct 
    language 
from 
    reviews_2024

### Snowflake Cortex

Snowflake Cortex is an intelligent, fully managed service that hosts and serves top-performing LLMs and vector functions which quickly and securely process text and build AI applications contextualized with  data stored in unstructured and semi-structured formats. We can access it via serverless SQL / Python functions. 
The available capabilities include:
- Perform complex reasoning tasks with your text data using COMPLETE.
- Extract information from unstructured or semi-structured data using EXTRACT_ANSWER.
- Automatically get a synopsis of text with SUMMARIZE.
- Detect the mood or tone of text using SENTIMENT.
- Translate documents to other languages using TRANSLATE.




In the next cell, we leverage **Translate** - one of the **Snowflake Cortex specialised LLM functions** are available in Snowpark ML, to translate the multilingual reviews to english to enable easier analysis for folks who don't speak the language the original review is in.

In [ ]:
create or replace table reviews_2024 as 
select 
    *, 
    case 
        when language != 'en' then snowflake.cortex.translate(review, language, 'en')
        else review 
    end as translated_review
from 
    reviews_2024

In [ ]:
select * from reviews_2024 where language != 'en' limit 100

In the following cell, we utilize **Snowflake Cortex - Complete** function to understand at scale based on a given customer's review what rating that customer is likely to rate their experience at the food truck. 

We provide a prompt with the instruction of the task along with a sample example of the completed task also known as **one-shot learning**. This prompt is then provided to complete function along with the review. 

In [ ]:
prompt = """[INST]
### 
You are tasked with rating cutsomer reviews for global food truck network called tasty bytes. \
Rating can be one of the following - awful, poor, okay, good, excellent such that awful is the worst \
possible rating and excellent is the best possible rating. Include only the rating in the output \
without any additional text. \
Rate the following review:
The "Freezing Point" ice cream truck in Seoul offered a mix of classic and unique \
options. The Sugar Cone and Waffle Cone were both fresh and crisp, providing a satisfying crunch. The \
Bottled Water and Ice Tea were standard, as expected. The standout, however, was the Mango Sticky Rice \
- a delightful blend of sweet and tangy, it was a refreshing twist to the usual ice cream flavors. The \
service was efficient, but lacked a personal touch. Overall, it\'s a decent choice for a quick, cool \
treat in the city.
Rating : good
Rate the following review: 
###"""

In [ ]:
-- try different models
-- mixtral-8x7b, minstral-large, llama3-70b, llama3-8b
select 
    *,
    snowflake.cortex.complete('mixtral-8x7b',
                              concat_ws('\n', $${{prompt}}$$, review, '[/INST]')
    ) as rating,
    case 
        when rating ilike 'awful' then 'awful'
        when rating ilike 'poor' then 'poor'
        when rating ilike 'okay' then 'okay'
        when rating ilike 'good' then 'good'
        when rating ilike 'excellent' then 'excellent'
        else 'unsure' end as clean_rating
from 
    reviews_2024
limit 100;

In [ ]:
create or replace table reviews_2024 as 
select 
    *,
    snowflake.cortex.complete('llama3-8b',
                              concat_ws('\n', $${{prompt}}$$, review, '[/INST]')
    ) as rating,
    case 
        when rating ilike 'awful' then 'awful'
        when rating ilike 'poor' then 'poor'
        when rating ilike 'okay' then 'okay'
        when rating ilike 'good' then 'good'
        when rating ilike 'excellent' then 'excellent'
        else 'unsure' end as clean_rating
from 
    reviews_2024


We can similarly understand if a customer would recommend the food truck based on their review using Snowflake Cortex complete. 

This time we don't provide any sample examples for the task within the prompt just the instruction - zero shot learning. Complete does a fairly good job at the required task.

In [ ]:
select 
    *,
    snowflake.cortex.complete('llama3-8b',
                              concat_ws('\n',
                                        $$
        [INST]
            ### 
            Tell me based on the following food truck customer review, will they recommend the food truck to \
            their friends and family? Answer should be only one of the following words - \
            "Likely" or "Unlikely" or "Unsure". Make sure there are no additional additional text.
            Review -
            ###$$,
        review )
    ) as recommend,
    case
        when recommend ilike 'unlikely' then 'Unlikely'
        when recommend ilike 'likely' then 'Likely'
        else 'Unsure'
    end as recommend_clean
from 
reviews_2024
limit 100;

In [ ]:
create or replace table reviews_2024 as
select 
    *,
    snowflake.cortex.complete('llama3-8b',
                              concat_ws('\n',
                                        $$
        [INST]
            ### 
            Tell me based on the following food truck customer review, will they recommend the food truck to \
            their friends and family? Answer should be only one of the following words - \
            "Likely" or "Unlikely" or "Unsure". Make sure there are no additional additional text.
            Review -
            ###$$,
        review )
    ) as recommend,
    case
        when recommend ilike 'unlikely' then 'Unlikely'
        when recommend ilike 'likely' then 'Likely'
        else 'Unsure'
    end as recommend_clean
from 
reviews_2024

In the above cells, we called the  LLM function on a Snowpark Dataframe column and created new columns which enabling us to use the insights delivered by Cortex in our data exploration. 

We can see this action in the upcoming cells where we exploring the distribution of likelihood of recommendation across different truck brands and trucks using Snowpark Dataframe.

In [ ]:
with brand_recommend_summ as (
    select 
        truck_brand_name,
        recommend_clean,
        count(1) as ct
    from 
        reviews_2024
    group by 
        truck_brand_name, 
        recommend_clean
) 
select 
    *, 
    ct / sum(ct) over (partition by truck_brand_name) as pct_of_brand
from 
    brand_recommend_summ
order by 1, 2

In [ ]:
st.bar_chart(RECOMMEND_BRAND_DIST.to_pandas(), x="TRUCK_BRAND_NAME", y="PCT_OF_BRAND", color="RECOMMEND_CLEAN")

We see above that Kitkat Ramen Bar has a high proportion of customers who are unlikely to recommend their food trucks. Next, we dive deeper to see which specific trucks within Kitkat Ramen Bar may be behind this issue. 

In [ ]:
with brand_recommend_summ as (
    select 
        truck_id,
        recommend_clean,
        count(1) as ct
    from 
        reviews_2024
    where 
        truck_brand_name = 'Kitakata Ramen Bar'
    group by 
        truck_id, 
        recommend_clean
) 
select 
    *, 
    ct / sum(ct) over (partition by truck_id) as pct_of_truck
from 
    brand_recommend_summ
order by 1, 2

In [ ]:
pct_recommend = RAMEN_TRUCK_DETAIL.to_pandas()
pct_recommend = pct_recommend[pct_recommend["RECOMMEND_CLEAN"] == "Likely"]
st.bar_chart(pct_recommend, x="TRUCK_ID", y="PCT_OF_TRUCK")

Trucks apart from Truck 5 seem to have a healthy proportion of likely to unlikely to recommend customers. Further down in the notebook, we will look at the reason why the customers are not likely to recommend Truck 5 to others.

So far we saw Snowflake Cortex - Translate & Complete. Next we will look at another **task specific LLM function in Cortex - Sentiment**. We utilise sentiment function to understand customer's tone based on the review they provided. Sentiment return value between -1 and 1 such that -1 is the most negative while 1 is the most positive.  

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.Sentiment(F.col('REVIEW')))

reviews_df.select(["REVIEW","SENTIMENT"]).show(3)

In [ ]:
create or replace table reviews_2024 as 
select 
    *,
    snowflake.cortex.sentiment(review) as sentiment, 
    case 
        when sentiment > 0.3 then 'Positive'
        when sentiment < -0.3 then 'Negative'
        else 'Neutral' 
    end as sentiment_grp
from 
    reviews_2024;
    
select * from reviews_2024 limit 5;

Next, we visualise sentiment across time and can see a healthy distribution of positive, negative & neutral reviews. 

In [ ]:
with sentiment_by_date as 
(select 
        date, 
        sentiment_grp, 
        count(1) as ct 
    from 
        reviews_2024
    group by 1, 2)
select 
    *, 
    ct / sum(ct) over (partition by date) as pct 
from 
    sentiment_by_date

In [ ]:
st.area_chart(SENTIMENT_BY_DATE.to_pandas(), x="DATE", y="PCT", color="SENTIMENT_GRP")

We can take our analysis a step further by looking at aspect based sentiment instead just the overall sentiment of review and understand what the customers thinks about different aspects like food quality, service, pricing etc. 
We do so by leveraging Snowflake cortex Complete coupled with a prompt that includes one shot example. 

In [ ]:
# Prompt to understand sentiment for different categories mentioned in the customer review
# We employ one shot incontext learning to inform LLM
prompt = """[INST]
### 
You are analyzing food-truck customer reviews to undertsand what a given review says about different relevant categories like \
food quality, menu options, staff, overall experience, price, ambience, customer support, \
hygiene standards etc and if sentiment is negative,positive or neutral for that category. \
Only answer in a single valid JSON containing "category", "sentiment" and "details". \
Make sure there is no additional text and not mention categories in answer which are not \
talked in the review. \
Get category based sentiment for the follwoing customer review:
"This food truck offers a disappointing experience. \
The menu lacks healthy options and the food quality is subpar. Finding a parking spot near the \
truck can also be a frustrating ordeal. Additionally, the value for money is not worth it. To top \
it all off, the service provided at this food truck is less than pleasant, adding to the overall \
negative dining experience. Tried reaching out the customer support but was unable to get through." 
Answer : [{     "category": "food quality",     "sentiment": "negative",    "details": "subpar quality"   }, {     "category": "menu options",     "sentiment": "negative",     "details": "lacks healthy options"   },   {     "category": "staff",     "sentiment": "negative",     "details": "unpleasant"   },   {     "category": "price",     "sentiment": "negative",     "details": "not worth the money"   },   {     "category": "experience",     "sentiment": "negative",     "details": "regrettable dining experience"   },   {     "category": "customer support",     "sentiment": "negative",     "details": "unable to get through"   } ].
Get category based sentiment for the follwoing customer review:
###"""

# Ask Cortex Complete and create a new column
reviews_df = session.table('REVIEWS_2024')
reviews_df = reviews_df.withColumn('CATEGORY_SENTIMENT', cortex.Complete('mixtral-8x7b', \
                                                            F.concat(F.lit(prompt), \
                                                                     F.col('REVIEW'), \
                                                                     F.lit("""Answer:[/INST]"""))))
reviews_df.select(["REVIEW","CATEGORY_SENTIMENT"]).show(1)

Using the sentiment we derieved and Complete, we aggregate the top most negative reviews for tasty bytes and understand what are the main issues plaguing the food truck network.

We can see the 3 main issues - 
- Poor Food Quality
- Unresponsive Customer Support
- Unacceptable Service


In [ ]:
# Aggregrate the 100 most negative reviews for tasty bytes
reviews_agg_ = reviews_df.order_by(F.col('SENTIMENT')).select(F.col('REVIEW')).first(100)

reviews_agg_str = ''.join(map(str,reviews_agg_))

# Prompt to summarize the three top issues flagged in the aggregated reviews
prompt = """[INST]###Summarize the issues mentioned in following aggregated food truck customer reviews with three \
concise bullet points under 50 words each such that each bullet point also has a heading along with \
recommendations to remedy those issues.###""" + reviews_agg_str + """[/INST]"""

# Answer from Cortex Complete
print(cortex.Complete('mistral-large', prompt))

We utilize sentiment next to understand which truck has the most negative reviews on average. This analysis confirms Truck 5 to be the most negatively reviewed truck. 

In [ ]:
# Get average sentiment of reviews for Trucks 
truck_agg_reviews_df = reviews_df.groupBy(F.col('TRUCK_ID')) \
                .agg(F.avg(F.col('SENTIMENT')).alias('AVG_SENTIMENT'),F.count(F.col('REVIEW_ID')).alias('REVIEW_COUNT')) 
truck_agg_reviews_df.show(3)

In [ ]:
# Get the truck with most negative average sentiment
truck_agg_reviews_df.filter(F.col('REVIEW_COUNT') >= 10).order_by(F.col('AVG_SENTIMENT')) \
                .select(F.col('TRUCK_ID')).limit(1).collect()[0][0]

We can quickly analysis the most negative reviews for Truck 5 to understand the main issues that the customers complain about by leveraging Snowflake cortex Complete.

In [ ]:
# Aggregate the most negative reviews for Truck 5
reviews_agg_ = reviews_df.filter(F.col('TRUCK_ID') == MOST_NEGATIVELY_REVIEWED_TRUCK)\
               .order_by(F.col('SENTIMENT')).select(F.col('REVIEW')).first(100)

reviews_agg_str = ''.join(map(str,reviews_agg_))

# Prompt to understand the main issues with Truck 5
prompt = """[INST]###Summarize three main issues mentioned in following aggregated customer review with three concise bullet 
points under 50 words each such that each bullet point also has a heading.###""" + reviews_agg_str + """[/INST]"""

# Print Cortex Complete's answer
print(cortex.Complete('mistral-large', prompt))

We can ask Snowflake cortex Complete to draft an email which summarizes the issues at Truck 5 along with any recommendation to remedy them.

In [ ]:
# Prompt to get an email draft which reports the main issues with Truck 5 with recommendations to solve
prompt =""" [INST]### Write me survey report email to the franchise owner summarizing the issues mentioned in following \
aggregated customer review with three concise bullet points under 50 words each such that each bullet \
point also has a heading along with recommendations to remedy those issues.###"""+ reviews_agg_str +""" \
Mention the truck brand name and location in the email.[/INST]"""

# Print the result from Cortex Complete
print(cortex.Complete('mixtral-8x7b', prompt))